In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_dir = './drive/MyDrive/Crop_Image_Dataset'

In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

In [ ]:
class_names = os.listdir(source_dir)
source_dir = './drive/MyDrive/Crop_Image_Dataset'
train_dir = "/content/drive/My Drive/path/to/train"
val_dir = "/content/drive/My Drive/path/to/validation"
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    seed=42,
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
)

Found 29006 images belonging to 27 classes.
Found 7160 images belonging to 27 classes.


In [ ]:


os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for class_name in class_names:
    class_source_dir = os.path.join(source_dir, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)

    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    image_files = os.listdir(class_source_dir)

    if len(image_files) > 0:
        train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=42)

        for image in train_images:
            src = os.path.join(class_source_dir, image)
            dst = os.path.join(train_class_dir, image)
            try:
                shutil.copy(src, dst)
                print(f"Moved '{image}' to training set in class '{class_name}'")
            except Exception as e:
                print(f"Failed to copy '{image}' to training set in class '{class_name}': {str(e)}")

        for image in val_images:
            src = os.path.join(class_source_dir, image)
            dst = os.path.join(val_class_dir, image)
            try:
                shutil.copy(src, dst)
                print(f"Moved '{image}' to validation set in class '{class_name}'")
            except Exception as e:
                print(f"Failed to copy '{image}' to validation set in class '{class_name}': {str(e)}")
    else:
        print(f"Warning: No images found in class '{class_name}'")


Moved 'bbf0af8f-62fc-4b41-9b4a-e1f7e15974c7___JR_FrgE.S 2788_new30degFlipLR_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved '9eaff91a-1260-4d17-844d-65ab1ee09752___JR_FrgE.S 2736_new30degFlipLR_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved '6af653b0-6a01-458c-bb17-5ee1b76115f4___JR_FrgE.S 2753_270deg(1)_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved 'a0d95497-da4c-40dd-bc2a-d29d1629bd4f___JR_FrgE.S 8759_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved '8381c748-f548-4ae2-be56-cb72184a7055___JR_FrgE.S 3082_new30degFlipLR_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved '47800fc6-b681-46ab-aec3-01076d650c5d___JR_FrgE.S 3051_new30degFlipLR(1)_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved '7530c184-6aa8-40fa-9f8d-7e5f0b3e5a7e___JR_FrgE.S 3045_new30degFlipLR_BGremoved.JPG' to training set in class 'Apple___Black_rot'
Moved '4e6311ab-8927-4147-9857-a669f2268cb1___JR_FrgE.S 8726_270d

KeyboardInterrupt: ignored

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = keras.Sequential([
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(27, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint('your_model.h5', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_data, validation_data=val_data, epochs=7, callbacks=[checkpoint, early_stopping])

Epoch 1/7
1813/1813 [==============================] - 6083s 3s/step - loss: 0.7444 - accuracy: 0.7498 - val_loss: 0.5734 - val_accuracy: 0.8487
Epoch 2/7


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1813/1813 [==============================] - 434s 240ms/step - loss: 0.4009 - accuracy: 0.8695 - val_loss: 0.5787 - val_accuracy: 0.8568
Epoch 3/7
1813/1813 [==============================] - 436s 240ms/step - loss: 0.3037 - accuracy: 0.9008 - val_loss: 0.3348 - val_accuracy: 0.9091
Epoch 4/7
1813/1813 [==============================] - 433s 239ms/step - loss: 0.2581 - accuracy: 0.9154 - val_loss: 0.4832 - val_accuracy: 0.8870
Epoch 5/7
1813/1813 [==============================] - 434s 239ms/step - loss: 0.2341 - accuracy: 0.9241 - val_loss: 0.2495 - val_accuracy: 0.9346
Epoch 6/7
1813/1813 [==============================] - 436s 240ms/step - loss: 0.2216 - accuracy: 0.9280 - val_loss: 0.2745 - val_accuracy: 0.9242
Epoch 7/7
1813/1813 [==============================] - 447s 246ms/step - loss: 0.1913 - accuracy: 0.9395 - val_loss: 0.3488 - val_accuracy: 0.9071
